# Data Preprocessing (Face Extraction)

## Download Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/Stanford/236/236 Dataset/2022-02-17.zip" -d "data"

## Face Extraction Based on Pre-trained Caffe Model

In [3]:
import os
import cv2
import numpy as np

def extract_face(f, required_size=(96, 96)):
  # Define paths
  prototxt_path = '/content/drive/MyDrive/Stanford/236/model_data/deploy.prototxt.txt'
  caffemodel_path = '/content/drive/MyDrive/Stanford/236/model_data/weights.caffemodel'

  # Read the model
  model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

  # Create directory 'faces' if it does not exist
  if not os.path.exists('faces'):
	  print("New directory created")
	  os.makedirs('faces')

  # Loop through all images and strip out faces
  count = 0
  file = f[5:]
  #f = "images/"+file

  file_name, file_extension = os.path.splitext(f)
  if (file_extension in ['.png','.jpg']):
    image = cv2.imread(f)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (200, 200), interpolation = cv2.INTER_AREA), 1.0, (200, 200), (104.0, 177.0, 123.0))
    model.setInput(blob)
    detections = model.forward()

		# Identify each face
    for i in range(0, 1):#detections.shape[2]):
      box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      (startX, startY, endX, endY) = box.astype("int")
      confidence = detections[0, 0, i, 2]

			# If confidence > 0.5, save it as a separate file
      if (confidence > 0.5):
        count += 1
        frame = image[startY:endY, startX:endX]
        if (frame.shape[0]<=75 or frame.shape[1]<=75):
          return False
        frame = cv2.resize(frame, required_size, interpolation = cv2.INTER_AREA)
        cv2.imwrite('faces/'+str(i) + '_' + file, frame)
      else:
        return False
  return True

Extract face for each image in folder

In [4]:
import glob
path = "data/*.*"
database = {}
for file in glob.glob(path):
  extract_face(file)

New directory created


Rename Files

In [7]:
path = "faces/*.*"
i = 0
for file in glob.glob(path):
  os.rename(file, "nfaces/"+str(i)+".jpg")
  i = i+1

Zip Folder

In [ ]:
!zip -r "/content/drive/MyDrive/Stanford/236/236 Dataset/data.zip" "/content/nfaces"